In [1]:
from typing import overload, Union, Optional, List

from sympy import latex
from sympy.core._print_helpers import Printable
from IPython.display import Latex


@overload
def print_sympy_entity(text: Optional[str] = None) -> None:
    ...

@overload
def print_sympy_entity(text: Optional[str] = None, expression: Optional[Union[str, Printable]] = None, *, raw: bool = True) -> None:
    ...

@overload
def print_sympy_entity(text: Optional[str] = None, *expressions: Union[str, Printable], raw: bool = True) -> None:
    ...

def print_sympy_entity(*args, **kwargs) -> None:
    text: Optional[str] = kwargs.get("text", None)
    expression: Optional[Union[str, Printable]] = kwargs.get("expression", None)

    if text is None and args:
        text = args[0]
    
    expressions: List[Union[str, Printable]] = \
        [expr for expr in args[1:] if isinstance(expr, (str, Printable))] \
        if len(args) > 1 else []

    if expression is not None:
        expressions.append(expression)

    if not expressions:
        if text is not None:
            display(Latex("$ \\text{" + text + "} $"))
        return
    
    is_raw: bool = kwargs.get("raw", True)
    
    final_expression: str = \
        (latex(expressions[0]) if is_raw else expressions[0]) \
        if len(expressions) == 1 else \
        r"\\".join(map(latex, expressions) if is_raw else expressions)
    final_expression = "$ " + final_expression + " $"

    if text is not None:
        final_expression = "$ \\text{" + text + "} $ " + final_expression

    display(Latex(final_expression))


In [2]:
import sympy
from sympy import Matrix, S, Symbol, symbols, I, zeros, eye
from sympy import simplify, expand, expand_complex, latex
import numpy as np
from IPython.display import Latex

# Практическое занятие 17
# Компьютерный практикум по алгебре на Python
## Матричные разложения: Холецкого, LDL, LU, QR. Жорданова форма.

### Задание 1.
Построить разложение Холецкого матриц
$$
M1=\left(
\begin{matrix}
1&-3&0\\
-3&-2&10\\
0&10&7
\end{matrix}
\right)
\quad
M2=\left(
\begin{matrix}
18&1 - 2I& -2\\
1 + 2I&4&-3I\\
-2&3I&5
\end{matrix}
\right)
$$
Проверить положительную определенность эрмитовой матрицы.

In [3]:
M1: Matrix = Matrix((
    (1, -3, 0),
    (-3, -2, 10),
    (0, 10, 7)
))
M2: Matrix = Matrix((
    (18, 1 - 2*I, -2),
    (1 + 2*I, 4, -3*I),
    (-2, 3*I, 5)
))

M1_chol = M1.cholesky(hermitian=False)
M1_chol.simplify()
print_sympy_entity("Разложение Холецкого матрицы M1:", M1_chol)

M2_chol = M2.cholesky()
M2_chol.simplify()
print_sympy_entity("Разложение Холецкого матрицы M2:", M2_chol)
print_sympy_entity(f"Матрица M2 {'' if M2.is_positive_definite else 'не '}положительно определенная")

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Задание 2.
Построить  LDL разложение для матриц Задания 1.

In [4]:
LM1, DM1 = M1.LDLdecomposition(hermitian=False)
LM2, DM2 = M2.LDLdecomposition()
for M in (LM1, DM1, LM2, DM2):
    M.simplify()

print_sympy_entity(f"M1 =", M1)
display(Latex("$ L_{M1} = " + latex(LM1) + " $"))
display(Latex("$ D_{M1} = " + latex(DM1) + " $"))
display(Latex("$ L_{M1}D_{M1}L_{M1}^T = " + latex(simplify(LM1 * DM1 * LM1.T)) + " $"))

print_sympy_entity(f"M2 =", M2)
display(Latex("$ L_{M2} = " + latex(LM2) + " $"))
display(Latex("$ D_{M2} = " + latex(DM2) + " $"))
display(Latex("$ L_{M2}D_{M2}L_{M2}^H = " + latex(simplify(LM2 * DM2 * LM2.H)) + " $"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Задание  3.
Построить  LU разложение для матрицы 
$$
V=\left(
\begin{matrix}
5&-2 - I&3 - 4I&1 + 4I\\
1 - I&-2&5 - I&2 - I\\
5&6 + I&0&5 
\end{matrix}
\right)
$$

In [5]:
V = Matrix((
    (5, -2 - I, 3 - 4*I, 1 + 4*I),
    (1 - I, -2, 5 - I, 2 - I),
    (5, 6 + I, 0, 5)
))

permutations: list[list[int]]
L, U, permutations = V.LUdecomposition()
for M in (L, U):
    M.simplify()

MLU = simplify((L * U).permuteBkwd(permutations))
print_sympy_entity(f"V =", V)
display(Latex("$ L_{V} = " + latex(L) + " $"))
display(Latex("$ U_{V} = " + latex(U) + " $"))
display(Latex("$ permutations = " + latex(permutations) + " $"))
display(Latex("$ LU.permuteBkwd(permutations) = " + latex(MLU) + " $"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Задание  4.
Построить  QR разложение для матрицы
$$
A=\left(
\begin{matrix}
3 + i&  2 & -i & 4 - 2i\\
-2 & -3 &  i & -3 + i\\
1 + i & -1 &  0 & 1 - i\\
-1 + i &  -4 & i & -2
\end{matrix}
\right)
$$
показать, что $A = QR$.

In [13]:
A: Matrix = Matrix((
    (3 + I, 2, -I, 4 - 2*I),
    (-2, -3, I, I - 3),
    (1 + I, -1, 0, 1 - I),
    (I - 1, -4, I, -2)
))

Q, R = A.QRdecomposition()
if Q.shape[1] > 2:
    Q = Q[:, :2]
if R.shape[0] > 2:
    R = R[:2, :]

for M in (Q, R):
    M.simplify()
QR: Matrix = simplify(Q * R)

print_sympy_entity("A =", A)
print_sympy_entity("Q = ", Q)
print_sympy_entity("R = ", R)
print_sympy_entity("matrix QR = ", QR)
print_sympy_entity(f"A == QR: {A == QR}")

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Задание  5.
Построить  жорданову форму для матрицы
$$
K=\left(
\begin{matrix}
6 &  5 & -2 & -3\\
-3 & -1 &  3 &  3\\
2 &  1 & -2 &  -3\\
-1 & 1 & 5 & 5
\end{matrix}
\right)
$$

In [7]:
K: Matrix = Matrix((
    (6, 5, -2, -3),
    (-3, -1, 3, 3),
    (2, 1, -2, -3),
    (-1, 1, 5, 5)
))

P, J = K.jordan_form()
for M in (P, J):
    M.simplify()

PJP_1 = simplify(P * J * P**(-1))

print_sympy_entity("K =", K)
print_sympy_entity("P =", P)
print_sympy_entity("J = ", J)
display(Latex("$ PJP^{-1} = " + latex(PJP_1) + " $"))
print_sympy_entity(expression=f"K == PJP^{{-1}}: {K == PJP_1}", raw=False)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Индивидуальное задание.
Решить с помощью  QR разложения матрицы
$A$
систему линейных уравнений
$AX = b$.

N50

$$A = \left[\begin{matrix}
    -8 & -8 & 6 & 4\\
    −6 & −6 & 7 & −4\\
    −29 & −24 & 26 & −5\\
    −23 & −18 & 19 & −1\\
\end{matrix}\right]$$

$$b = \left[\begin{matrix}
    −1 & −4 & −16 & −10
\end{matrix}\right]$$


In [8]:
A: Matrix = Matrix((
    (-8, -8, 6, 4),
    (-6, -6, 7, -4),
    (-29, -24, 26, -5),
    (-23, -18, 19, -1)
))
b: Matrix = Matrix((-1, -4, -16, 10))
Ab = A.row_join(b)
display(Latex(f"rk(A) = {latex(A.rank())}"))
display(Latex(f"rk((A | b)) = {latex(Ab.rank())}"))

X: Matrix = simplify(A.QRsolve(b)).col_join(Matrix([0]))
print_sympy_entity("X = ", X)
delta = A * X - b
print_sympy_entity("delta:", delta)
print_sympy_entity("норма разности:", delta.norm(2))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>